In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

# --- THIẾT LẬP ĐƯỜNG DẪN BAN ĐẦU ---
current_path = Path(os.getcwd())
PROJECT_ROOT = current_path.parent 
CITY = 'Paris'
SNAPSHOT_DATES = ['06 December, 2024', '03 March, 2025', '06 June, 2025', '12 September, 2025']

PROCESSED_DIR_INPUT = PROJECT_ROOT / 'processed' / CITY 
PROCESSED_DIR_OUTPUT = PROJECT_ROOT / 'processed'       
REPORTS_DIR = PROJECT_ROOT / 'reports'

# --- TÍNH TOÁN VÀ TỔNG HỢP (Không thay đổi) ---
all_listings_data = []

print("--- BẮT ĐẦU TÍNH TOÁN & TỔNG HỢP DỮ LIỆU ---")

for date_snapshot in SNAPSHOT_DATES:
    print(f"\n[SNAPSHOT] Đang tổng hợp: {date_snapshot}")
    snapshot_dir = PROCESSED_DIR_INPUT / date_snapshot
    
    try:
        df_listings = pd.read_csv(snapshot_dir / 'listings_processed.csv', low_memory=False)
        df_calendar = pd.read_csv(snapshot_dir / 'calendar_processed.csv', low_memory=False)
        df_reviews = pd.read_csv(snapshot_dir / 'reviews_processed.csv', low_memory=False)
        
        DAYS_IN_SNAPSHOT = df_calendar['date'].nunique()
        calendar_kpis = df_calendar.groupby('listing_id').agg(
            occupancy_rate=('is_booked', lambda x: x.sum() / DAYS_IN_SNAPSHOT),
            potential_revenue=('clean_price', lambda x: x.sum()) 
        ).reset_index()
        calendar_kpis.rename(columns={'listing_id': 'id'}, inplace=True)

        review_kpis = df_reviews.groupby('listing_id').agg(
            total_reviews_in_file=('id', 'count')
        ).reset_index()
        review_kpis.rename(columns={'listing_id': 'id'}, inplace=True)

        df_listings['id'] = df_listings['id'].astype(int) 
        df_final_snapshot = df_listings.merge(calendar_kpis, on='id', how='left')
        df_final_snapshot = df_final_snapshot.merge(review_kpis, on='id', how='left')
        df_final_snapshot['city'] = CITY
        
        all_listings_data.append(df_final_snapshot)
        print(f"  ✅ Tích hợp thành công. Số bản ghi: {len(df_final_snapshot):,}")

    except FileNotFoundError:
        print(f"  ⚠️ CẢNH BÁO: Không tìm thấy file processed cho {date_snapshot}. Bỏ qua.")
    except Exception as e:
        print(f"  ❌ LỖI trong quá trình xử lý {date_snapshot}: {e}")
        
if all_listings_data:
    valid_listings_data = [
        df for df in all_listings_data if df is not None and not df.empty and len(df.columns) > 0
    ]
    
    if valid_listings_data:
        df_combined = pd.concat(valid_listings_data, ignore_index=True)
        df_valid = df_combined[~df_combined['flag_price_outlier']].copy()
        print(f"\nSố bản ghi hợp lệ: {len(df_valid):,}")
    else:
        print("\nKhông có DataFrame hợp lệ nào để nối. Dừng script.")
        exit()
else:
    print("Không có dữ liệu để tổng hợp. Dừng script.")
    exit()

# =========================================================================
# --- TÍNH TOÁN VÀ LƯU CÁC FILE ĐẦU RA ---
# =========================================================================

print("\n--- Bắt đầu lưu 5 file đầu ra (Đã sửa tên KPI Khu Vực) ---")

# --- 0. LƯU FILE LISTINGS HỢP LỆ (Không tiền tố kpi_) ---
listings_output_path = PROCESSED_DIR_OUTPUT / 'listings_all_valid_paris.csv'
cols_to_keep = ['snapshot_date', 'id', 'price', 'latitude', 'longitude', 'neighbourhood_cleansed', 'room_type']
df_valid[cols_to_keep].to_csv(listings_output_path, index=False)
print(f"✅ 0. Đã lưu Listings hợp lệ: {listings_output_path}")


# --- 1. KPI VỀ GIÁ TỔNG QUÁT (kpi_price_paris.csv) ---
kpi_price_overall = df_valid.groupby('snapshot_date')['price'].median().reset_index()
kpi_price_overall.columns = ['snapshot_date', 'median_price']
output_path_price = PROCESSED_DIR_OUTPUT / 'kpi_price_paris.csv' 
kpi_price_overall.to_csv(output_path_price, index=False)
print(f"✅ 1. Đã lưu Giá tổng quát: {output_path_price}")

# --- 2. KPI VỀ NGUỒN CUNG TỔNG QUÁT (kpi_supply_paris.csv) ---
kpi_supply_overall = df_valid.groupby('snapshot_date').size().reset_index(name='total_listings')
output_path_supply = PROCESSED_DIR_OUTPUT / 'kpi_supply_paris.csv' 
kpi_supply_overall.to_csv(output_path_supply, index=False)
print(f"✅ 2. Đã lưu Nguồn cung: {output_path_supply}")

# --- 3. KPI VỀ CẤU TRÚC PHÒNG (kpi_room_type_paris.csv) ---
kpi_room_type_detailed = df_valid.groupby(['snapshot_date', 'room_type']).size().reset_index(name='count')
kpi_room_total = kpi_room_type_detailed.groupby('snapshot_date')['count'].transform('sum')
kpi_room_type_detailed['total'] = kpi_room_total
kpi_room_type_detailed['percentage'] = (kpi_room_type_detailed['count'] / kpi_room_type_detailed['total']) * 100
kpi_room_type_detailed['percentage'] = kpi_room_type_detailed['percentage'].round(8) 
kpi_room_final = kpi_room_type_detailed[['snapshot_date', 'room_type', 'count', 'total', 'percentage']]
output_path_room_type = PROCESSED_DIR_OUTPUT / 'kpi_room_type_paris.csv' 
kpi_room_final.to_csv(output_path_room_type, index=False)
print(f"✅ 3. Đã lưu Cấu trúc phòng: {output_path_room_type}")

# --- 4. GIÁ CHI TIẾT THEO KHU VỰC (price_nhood_paris.csv) ---
kpi_price_by_neighbourhood = df_valid.groupby(['snapshot_date', 'neighbourhood_cleansed']).agg(
    median_price=('price', 'median'),
    count=('id', 'count')
).reset_index()

# TÊN FILE ĐÃ ĐƯỢC ĐỔI: BỎ TIỀN TỐ "kpi_"
output_path_nhood_price = PROCESSED_DIR_OUTPUT / 'price_nhood_paris.csv'
kpi_price_by_neighbourhood.to_csv(output_path_nhood_price, index=False)
print(f"✅ 4. Đã lưu Giá theo khu vực: {output_path_nhood_price}")

print("\n--- HOÀN TẤT NOTEBOOK PARIS_SUMMARY_KPI.ipynb ---")

--- BẮT ĐẦU TÍNH TOÁN & TỔNG HỢP DỮ LIỆU ---

[SNAPSHOT] Đang tổng hợp: 06 December, 2024
  ✅ Tích hợp thành công. Số bản ghi: 91,031

[SNAPSHOT] Đang tổng hợp: 03 March, 2025
  ✅ Tích hợp thành công. Số bản ghi: 86,064

[SNAPSHOT] Đang tổng hợp: 06 June, 2025
  ✅ Tích hợp thành công. Số bản ghi: 84,055

[SNAPSHOT] Đang tổng hợp: 12 September, 2025
  ✅ Tích hợp thành công. Số bản ghi: 81,853


C:\Users\Admin\AppData\Local\Temp\ipykernel_34064\478870626.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat(valid_listings_data, ignore_index=True)



Số bản ghi hợp lệ: 169,131

--- Bắt đầu lưu 5 file đầu ra (Đã sửa tên KPI Khu Vực) ---
✅ 0. Đã lưu Listings hợp lệ: d:\btlltxldl\2526-LTXLDL-Project4-AIT2006-2-4.4\processed\listings_all_valid_paris.csv
✅ 1. Đã lưu Giá tổng quát: d:\btlltxldl\2526-LTXLDL-Project4-AIT2006-2-4.4\processed\kpi_price_paris.csv
✅ 2. Đã lưu Nguồn cung: d:\btlltxldl\2526-LTXLDL-Project4-AIT2006-2-4.4\processed\kpi_supply_paris.csv
✅ 3. Đã lưu Cấu trúc phòng: d:\btlltxldl\2526-LTXLDL-Project4-AIT2006-2-4.4\processed\kpi_room_type_paris.csv
✅ 4. Đã lưu Giá theo khu vực: d:\btlltxldl\2526-LTXLDL-Project4-AIT2006-2-4.4\processed\price_nhood_paris.csv

--- HOÀN TẤT NOTEBOOK PARIS_SUMMARY_KPI.ipynb ---
